In [1]:
import cv2
cv2.__version__

'3.4.1'

In [2]:
import sys
print(sys.version)

3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 16:13:55) [MSC v.1900 64 bit (AMD64)]


In [3]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk
import time
import pandas as pd

import skimage.io as io

import os

import imageio
# print environment
os.path.dirname(sys.executable)

'C:\\Users\\calli\\Anaconda3\\envs\\opencv3'

In [4]:
#vidPath = "D:/BeeTwoFlowerExperiment/2018_02_27__14_04_52_556/2018_02_27__14_04_55_680_cam1-0000.avi"
vidPath = r"D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52_556\2018_02_27__14_04_55_680_cam1-0000.avi"
os.path.isfile(vidPath)

True

In [5]:
cap = cv2.VideoCapture(vidPath)


length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)
cap.release()

546 1280 1024 10.0


In [6]:
def load_imgs(vidPath, start, stop):
    cap = cv2.VideoCapture(vidPath)
    imgs = []
    for ff in range(length):
        
        
        ret, frame = cap.read()
        
        if np.mod(ff, 10) == 0:
            print(ff)
        

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        imgs.append(img)
        
        if ff > stop:
            break
            
    cap.release()
    return imgs

In [7]:
imgs = load_imgs(vidPath, 0, 50)

0
10
20
30
40
50


In [8]:
io.imshow(imgs[0])

In [11]:
img = imgs[0]
img = cv2.cvtColor(imgs[0].copy(), cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img,5)
cimg = img.copy()

circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,5, 2000)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

cv2.imshow('detected circles',cimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = imgs[0]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray,5)

    
    
rows = gray.shape[0]
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, rows / 8,
                           param1=100, param2=30,
                           minRadius=1, maxRadius=300)


if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        cv2.circle(img, center, 1, (0, 100, 100), 3)
        # circle outline
        radius = i[2]
        cv2.circle(img, center, radius, (255, 0, 255), 3)


cv2.imshow("detected circles", img)
cv2.waitKey(0)

In [ ]:
io.imshow(gray)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape

In [12]:
import matplotlib.pyplot as plt

from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter

In [22]:
# Load picture, convert to grayscale and detect edges
image_rgb = imgs[0].copy()
image_gray = color.rgb2gray(image_rgb)
edges = canny(image_gray, sigma=1.0,
              low_threshold=0.55, high_threshold=0.8)

# Perform a Hough Transform
# The accuracy corresponds to the bin size of a major axis.
# The value is chosen in order to get a single high accumulator.
# The threshold eliminates low accumulators
result = hough_ellipse(edges, accuracy=20, threshold=250,
                       min_size=100, max_size=120)
result.sort(order='accumulator')

# Estimated parameters for the ellipse
best = list(result[-1])
yc, xc, a, b = [int(round(x)) for x in best[1:5]]
orientation = best[5]

# Draw the ellipse on the original image
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
image_rgb[cy, cx] = (0, 0, 255)
# Draw the edge (white) and the resulting ellipse (red)
edges = color.gray2rgb(img_as_ubyte(edges))
edges[cy, cx] = (250, 0, 0)

fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4),
                                sharex=True, sharey=True)

ax1.set_title('Original picture')
ax1.imshow(image_rgb)

ax2.set_title('Edge (white) and result (red)')
ax2.imshow(edges)

plt.show()

IndexError: index -1 is out of bounds for axis 0 with size 0

In [54]:

result

array([],
      dtype=[('accumulator', '<i8'), ('yc', '<f8'), ('xc', '<f8'), ('a', '<f8'), ('b', '<f8'), ('orientation', '<f8')])

In [79]:

plt.clf()
img = imgs[0]
io.imshow(img)
img = color.rgb2gray(img)
img.shape


(1024, 1280)

In [80]:
img = imgs[0]
img = color.rgb2gray(img)
edges = canny(img, sigma=0.4, low_threshold=0.1, high_threshold=0.7)

plt.clf()
io.imshow(edges*1.0)

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "


In [91]:
plt.clf()
io.imshow(edges*1.0)

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "


In [96]:
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

img = imgs[0]
img = color.rgb2gray(img)
edges = canny(img, sigma=0.4, low_threshold=0.1, high_threshold=0.7)


# Detect two radii
hough_radii = np.arange(120, 150, 2)
hough_res = hough_circle(edges, hough_radii)

# Select the most prominent 5 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=3)

In [97]:
radii


array([142, 142, 132])

In [98]:
# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
img = color.gray2rgb(img)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    img[circy, circx] = (220, 20, 20)

ax.imshow(img, cmap=plt.cm.gray)
plt.show()

In [99]:
io.imshow(img)

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:80: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warn("Float image out of standard range; displaying "
